In [1]:
!pip install -q torch==2.1.0 --index-url https://download.pytorch.org/whl/cu121

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 2.1.0+cu121 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 2.1.0+cu121 which is incompatible.
torchvision 0.18.1+cu121 requires torch==2.3.1, but you have torch 2.1.0+cu121 which is incompatible.


In [2]:
!pip install -q "transformers==4.34.0" "datasets==2.13.0" "peft==0.6.2" "accelerate==0.21.0" "bitsandbytes==0.41.2.post2" "trl==0.4.7" "safetensors>=0.3.1" "gdown==4.6"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00


In [3]:
from huggingface_hub import login
from google.colab import userdata

In [4]:
login(token=userdata.get('hf_read_chelvan'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config=bnb_config,
    device_map="cuda:0",
    trust_remote_code=True
)
model.config.use_cache = False

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Preparing Dataset

In [6]:
!gdown --id 1zfjg_3viiQ1IOR4AbsKiT4QCY1I1qWg8

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1zfjg_3viiQ1IOR4AbsKiT4QCY1I1qWg8
To: /content/Pandas_Code_SFT_Dataset.csv
100% 77.8k/77.8k [00:00<00:00, 3.24MB/s]


In [7]:
import pandas as pd
from datasets import Dataset

dataset_df = pd.read_csv('Pandas_Code_SFT_Dataset.csv')
df = dataset_df.sample(frac=1).reset_index(drop=True)
print(df.columns)

Index(['Dataset Name', 'Columns with Sample Data', 'Question',
       'Expected Code'],
      dtype='object')


In [8]:
df.head(5)

,Dataset Name,Columns with Sample Data,Question,Expected Code
0,books,"data = {\n ""book_id"": [301, 302, 303, 304, ...",Find the top 3 oldest books in terms of public...,```\ndef process(df):\n # Sort the DataFram...
1,sales,"data = {\n ""invoice_date"": [""2023-09-01"", ""...",How many invoices were generated in the last q...,```\ndef process(df):\n # Convert the 'invo...
2,patient,"data = {\n ""patient_id"": [1001, 1002, 1003,...",Identify the male patients with Heart Disease ...,```\ndef process(df):\n from datetime impor...
3,patient,"data = {\n ""patient_id"": [1001, 1002, 1003,...",Which patients are prescribed a dosage higher ...,```\ndef process(df):\n # Function to conve...
4,department,"data = {\n ""employee_id"": [101, 102, 103, 1...",Identify the employee with the highest salary ...,```\ndef process(df):\n # Filter for employ...


In [9]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['Dataset Name', 'Columns with Sample Data', 'Question', 'Expected Code'],
    num_rows: 52
})

In [10]:
def apply_prompt_template(examples):
  instruction = examples['Question']
  context = examples["Columns with Sample Data"]
  response = examples['Expected Code']

  full_prompt = f"""Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
{instruction}

header columns with sample data:
{context}

### Response:
{response}

### End
"""
  return { "text": full_prompt }

dataset = dataset.map(apply_prompt_template)
print(dataset["text"][0])

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
Find the top 3 oldest books in terms of publication year and list their titles, authors, and genres?

header columns with sample data:
data = {
    "book_id": [301, 302, 303, 304, 305],
    "title": ["The Mystery of Time", "Journey Through Stars", "Secrets of the Ocean", "The Lost Civilization", "Worlds Beyond"],
    "author": ["Alice Johnson", "John Smith", "Emily White", "David Brown", "Sarah Green"],
    "genre": ["Mystery", "Science Fiction", "Adventure", "Historical", "Fantasy"],
    "publication_year": [2018, 2020, 2019, 2017, 2021],
    "pages": [320, 250, 400, 500, 300],
    "borrowed_times": [25, 15, 35, 20, 10],
    "available_copies": [2, 3, 1, 4, 5]
}

### Response:
```
def process(df):
    # Sort the DataFrame by publication year in ascending order to find the oldest books
    oldest_books = df.sort_values(by='publication_year').head

In [11]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'k_proj', 'v_proj'] # Choose all linear layers from the model
)

In [12]:
from transformers import TrainingArguments


training_params = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=50,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Train

In [13]:
from trl import SFTTrainer


trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_params,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:136: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/52 [00:00<?, ? examples/s]

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,0.902700
2,0.992800
3,1.201300
4,0.667000
5,0.887100
6,0.570300
7,0.851600
8,0.419500
9,0.384400
10,0.410900


TrainOutput(global_step=50, training_loss=0.321178385540843, metrics={'train_runtime': 711.9554, 'train_samples_per_second': 0.281, 'train_steps_per_second': 0.07, 'total_flos': 4982587941126144.0, 'train_loss': 0.321178385540843, 'epoch': 3.85})

In [14]:
from transformers import pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

# Test

In [15]:
prompt = """
Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
how many unique items are there

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
"""


result = pipe(prompt, max_length=550)
generated_text = result[0]['generated_text']
print(generated_text.split("### End")[0])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
how many unique items are there

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
```
def process(df):
    # Count the 

In [16]:
prompt = """
Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
top 3 products has higest discounts

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
"""


result = pipe(prompt, max_length=550)
generated_text = result[0]['generated_text']
print(generated_text.split("### End")[0])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
top 3 products has higest discounts

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
```
def process(df):
    # Calcul

In [17]:
prompt = """
Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
Which item has customer C006 spent the most money on

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
"""


result = pipe(prompt, max_length=550)
generated_text = result[0]['generated_text']
print(generated_text.split("### End")[0])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
Which item has customer C006 spent the most money on

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
```
def process(

# Push fine-tuned model to hub

In [18]:
login(token=userdata.get('niru_hf_write'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [19]:
trainer.save_model("Llama-3-8B_SFT_Finetune_Pandas_Code")

In [20]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/Llama-3-8B_SFT_Finetune_Pandas_Code",
    low_cpu_mem_usage=True,
)

# Merge LoRA and base model
merged_model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [21]:
# push merged model to the hub
merged_model.push_to_hub("Llama-3-8B_SFT_Finetune_Pandas_Code",private=True,safe_serialization=True)
tokenizer.push_to_hub("Llama-3-8B_SFT_Finetune_Pandas_Code",private=True)

model-00004-of-00004.safetensors:   0%|          | 0.00/2.34G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/9.83G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nirusanan/Llama-3-8B_SFT_Finetune_Pandas_Code/commit/de8529a5f2f3073c02605b00d38838c4089047cc', commit_message='Upload tokenizer', commit_description='', oid='de8529a5f2f3073c02605b00d38838c4089047cc', pr_url=None, pr_revision=None, pr_num=None)